<a href="https://colab.research.google.com/github/LeeSeungwon89/Deep-learning_Theory/blob/main/8-2%20%ED%95%A9%EC%84%B1%EA%B3%B1%20%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%84%20%EC%82%AC%EC%9A%A9%ED%95%9C%20%EC%9D%B4%EB%AF%B8%EC%A7%80%20%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **8-2 합성곱 신경망을 사용한 이미지 분류**

## **패션 MNIST 데이터 불러오기**

패션 MNIST 데이터를 준비하겠습니다. 다만 이전 챕터에서 수행했던 전처리 과정과 다른 부분이 있습니다. 완전 연결 신경망에서는 입력 이미지를 밀집층에 연결하기 위해 `reshape()` 메서드나 `Flatten` 클래스를 사용하여 일렬로 펼쳤습니다. 그러나 합성곱 신경망에서는 2차원 이미지를 바로 사용하므로 일렬로 펼치는 작업을 수행할 필요가 없습니다. 다만 입력 이미지는 깊이 차원(채널)이 있어야 합니다. 흑백 이미지는 채널 차원이 없는 2차원 배열이지만 `Conv2D` 층을 사용하려면 마지막에 채널 차원을 추가해야 합니다. `reshape()` 메서드를 사용하면 전체 배열 차원을 그대로 유지하면서 차원만 추가할 수 있습니다.

데이터를 준비하겠습니다.

In [1]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
# 기존 (48000, 28, 28) 3차원 배열을 (48000, 28, 28, 1) 4차원 배열로 차원을 바꿉니다.
# 데이터 스케일을 0 ~ 1 사이로 바꾸기 위해 255로 나눕니다.
train_scaled = train_input.reshape(-1, 28, 28, 1) / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target,
                                                                      test_size=0.2, random_state=42)

4431872/4422102 [==============================] - 0s 0us/step


## **합성곱 신경망 만들기**



---

작업 중

---



## **모델 컴파일과 훈련**